In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!git clone https://github.com/mirna-k/few-shot-learning-project.git
%cd few-shot-learning-project/

In [ ]:
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

from easyFSL_helper import *
from dataset_helper import *

from torch.utils.tensorboard import SummaryWriter

In [9]:
train_transform = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((180, 180)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
batch_size = 64
path = 'archive\CUB_200_2011'

N = 5
K = 5
n_query = 10
n_tasks_per_epoch = 100
n_validation_tasks = 50

train_set = CUB(split="train", training=True)
val_set = CUB(split="val", training=False)

print("Train: {}".format(len(train_set)))
print("Valid: {}".format(len(val_set)))

Train: 8245
Valid: 1780


In [5]:
train_sampler = TaskSampler(train_set, n_way=N, n_shot=K, n_query=n_query, n_tasks=n_tasks_per_epoch)
val_sampler = TaskSampler(val_set, n_way=N, n_shot=K, n_query=n_query, n_tasks=n_validation_tasks)

train_loader = data.DataLoader(
    train_set,
    batch_sampler=train_sampler,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)
val_loader = data.DataLoader(
    val_set,
    batch_sampler=val_sampler,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

In [19]:
resnet = models.resnet34()
few_shot_classifier = PrototypicalNetworks(resnet).to(device)

In [14]:
loss_module = nn.CrossEntropyLoss()
optimizer = optim.Adam(few_shot_classifier.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [7]:
import copy
def train_model(model, optimizer, scheduler, train_data_loader, loss_module, num_epochs=100, logging_dir='runs/our_experiment'):
    model_dir = logging_dir.split('/')[-1]
    os.makedirs(logging_dir, exist_ok=True)
    os.makedirs(f'models/{model_dir}', exist_ok=True)
    writer = SummaryWriter(logging_dir)

    best_validation_accuracy = 0.0

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        for support_images, support_labels, query_images, query_labels, _ in tqdm(train_data_loader, 'Epoch %d'%(epoch + 1)):

            model.process_support_set(support_images.to(device), support_labels.to(device))
            classification_scores = model(query_images.to(device))

            loss = loss_module(classification_scores, query_labels.to(device))

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()
            epoch_loss += loss.item() * len(support_images)

        epoch_loss /= len(train_data_loader.dataset)

        # Calling scheduler
        scheduler.step(epoch_loss)

        validation_accuracy, val_loss, _, _, f1= evaluate(model, loss_module, val_loader, device=device, tqdm_prefix="Validation")

        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_state = copy.deepcopy(model.state_dict())
            torch.save(model.state_dict(), f'models/{model_dir}/model{epoch + 1}.pt')
            print("Found a new best model.")

        writer.add_scalars('loss', {'train': epoch_loss,
                                    'val': val_loss}, epoch + 1)

        writer.add_scalar("Val_acc", validation_accuracy, epoch + 1)
        writer.add_scalar("F1", f1, epoch + 1)

    writer.close()

In [10]:
train_model(few_shot_classifier, optimizer, scheduler, train_loader, loss_module, num_epochs=100)

Epoch 1:   0%|          | 0/100 [00:00<?, ?it/s]

Validation:   0%|          | 0/50 [00:00<?, ?it/s]

Found a new best model.


Epoch 2:   0%|          | 0/100 [00:00<?, ?it/s]

Validation:   0%|          | 0/50 [00:00<?, ?it/s]

...

Epoch 99:   0%|          | 0/100 [00:00<?, ?it/s]

Validation:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch 100:   0%|          | 0/100 [00:00<?, ?it/s]

Validation:   0%|          | 0/50 [00:00<?, ?it/s]

Found a new best model.


In [ ]:
%tensorboard --logdir runs/our_experiment

In [10]:
batch_size = 128
dataset_path = 'animals_dataset'
labelmap_path = 'animals_label_map.json'

class_data = ClassData.get_class_data(dataset_path, labelmap_path, val_transform)
dataset = CustomDataset(class_data)

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
embeddings_df = predict_embeddings(data_loader, resnet, device=device)
features_dataset = FeaturesDataset.from_dataframe(embeddings_df)

In [12]:
task_sampler = TaskSampler(features_dataset, n_way=N, n_shot=K, n_query=n_query, n_tasks=100)

features_loader = DataLoader(features_dataset, batch_sampler=task_sampler, pin_memory=True, collate_fn=task_sampler.episodic_collate_fn)

In [27]:
few_shot_classifier = PrototypicalNetworks(resnet).to(device)
few_shot_classifier.load_state_dict(torch.load('models\our_experiment\model100.pt'))
few_shot_classifier = PrototypicalNetworks(backbone=nn.Identity())

accuracy, _, macro_precision, macro_recall, f1 = evaluate(few_shot_classifier, loss_module, features_loader, device="cpu")

print(f"Average accuracy : {(100 * accuracy):.6f} %\nMacro precision: {(100 * macro_precision):.6f}%\nMacro recall: {(100 * macro_recall):.6f}%\nF1: {f1}")

  0%|          | 0/100 [00:00<?, ?it/s]

Average accuracy : 26.080000 %
Macro precision: 25.990787%
Macro recall: 26.080000%
F1: 0.13603328000000003
